In [2]:
import firstStrategy

In [3]:
msg = firstStrategy.ConnectToStream()
endpoint = 'wss://stream.binance.com:9443/ws' 
open_position = False

In [4]:
#firstStrategy.SimulationTrade(msg, endpoint, usd=1000, open_position=False)

In [5]:
import websocket
import json
import pandas as pd

In [6]:
msg = json.dumps({"method": "SUBSCRIBE",
                  "params": ['ethusdt@kline_1m',
                            'ethusdt@kline_15m'],
                                "id":1})
endpoint = 'wss://stream.binance.com:9443/ws' 

In [19]:
endpoint = 'wss://stream.binance.com:9443/ws' 

msg = json.dumps({
"method": "SUBSCRIBE",
"params": ['ethusdt@kline_1m',
           'ethusdt@kline_15m'],
   "id":1})


returns = {'1m': 0, '15m':0}
open_position = False
usd = 1000

def on_open(ws): 
    ws.send(msg)

def on_message(ws, message):
    global df, buy_price, open_position, usd
    out = json.loads(message)
    df = pd.DataFrame(out['k'],index=[pd.to_datetime(out['E'], unit='ms')])[['s','i','o','c']]
    df['ret ' + df.i.values[0]] = float(df.c)/float(df.o) - 1
    returns[df.i.values[0]] = float(df.c)/float(df.o) - 1
    print(df)
    print(float(df.c))
    if not open_position and returns['15m'] < 0 and returns['1m'] > 0:
        buy_price = float(df.c)
        open_position = True 
        print('bought for ' + str(buy_price))
    if open_position:
        print('target profit: ' + str(buy_price * 1.002))
        print('stop loss: ' + str(buy_price * 0.998))
        print('USD: ' + str(usd))
        print('current price: ' + str(float(df.c)))
        if float(df.c) > buy_price * 1.002:
            #print("SOLD!")
            print('SOLD! profit made: ' + str((float(df.c) - buy_price)))
            open_position = False
            usd = usd * (((float(df.c) - buy_price) / 100) + 1)
        elif float(df.c) < buy_price * 0.998:
            #print("SOLD")
            print('LOSS STOPPED! loss: ' + str((float(df.c) - buy_price)))
            open_position = False
            usd = usd * (((float(df.c) - buy_price) / 100) + 1)

ws = websocket.WebSocketApp(endpoint, on_message=on_message, on_open=on_open)

ws.run_forever()


                               s    i              o              c   ret 15m
2023-10-20 12:54:16.494  ETHUSDT  15m  1609.99000000  1605.00000000 -0.003099
1605.0
                               s   i              o              c    ret 1m
2023-10-20 12:54:17.357  ETHUSDT  1m  1604.89000000  1605.00000000  0.000069
1605.0
bought for 1605.0
target profit: 1608.21
stop loss: 1601.79
USD: 1000
current price: 1605.0
                               s    i              o              c   ret 15m
2023-10-20 12:54:18.494  ETHUSDT  15m  1609.99000000  1605.00000000 -0.003099
1605.0
target profit: 1608.21
stop loss: 1601.79
USD: 1000
current price: 1605.0
                               s   i              o              c    ret 1m
2023-10-20 12:54:19.441  ETHUSDT  1m  1604.89000000  1605.00000000  0.000069
1605.0
target profit: 1608.21
stop loss: 1601.79
USD: 1000
current price: 1605.0
                               s    i              o              c   ret 15m
2023-10-20 12:54:20.494  ETHUSDT  

False